In [137]:
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import itertools

In [138]:
df = yf.download(tickers = 'ETH-USD', start = '2022-05-08', end = '2022-07-07', interval = '15m')
feed = bt.feeds.PandasData(dataname=df)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-05-07 22:00:00+00:00,2639.835205,2639.835205,2629.777832,2630.133057,2630.133057,141986816
2022-05-07 22:15:00+00:00,2630.287598,2631.183838,2623.864990,2625.324707,2625.324707,168380416
2022-05-07 22:30:00+00:00,2625.033203,2628.355713,2623.774414,2624.087891,2624.087891,175518720
2022-05-07 22:45:00+00:00,2623.761475,2623.761475,2599.523438,2604.839600,2604.839600,550936576
2022-05-07 23:00:00+00:00,2603.221680,2628.335205,2603.221680,2628.234619,2628.234619,469511168
...,...,...,...,...,...,...
2022-07-06 12:45:00+00:00,1136.847778,1136.847778,1134.137939,1134.137939,1134.137939,0
2022-07-06 13:00:00+00:00,1134.417603,1137.099854,1134.417603,1137.099854,1137.099854,3057664
2022-07-06 13:15:00+00:00,1137.330200,1142.304565,1137.330200,1141.688232,1141.688232,94081024


In [139]:
class EMACross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        stake=1,
        printout=False,
        onlylong=False,
        mtrade=False,
        stoporder=bt.Order.StopTrail
    )
    
    

    def __init__(self):
        ema1 = bt.ind.EMA(period=5)  # fast moving average
        ema2 = bt.ind.EMA(period=20)  # slow moving average
        self.crossover = bt.ind.CrossOver(ema1, ema2)  # crossover signal
        self.order = None
        
        if self.p.mtrade:
            self.tradeid = itertools.cycle([0, 1, 2])
        else:
            self.tradeid = itertools.cycle([0])
    

    def next(self):
        if self.order:
            return
        if self.crossover > 0:
            if self.position:
                self.close()
                print('CLOSE SHORT, %.2f' % self.data.close[0])
            self.buy()
            print('OPEN LONG, %.2f' % self.data.close[0])
        elif self.crossover < 0:
            if self.position:
                self.close()
                print('CLOSE LONG, %.2f' % self.data.close[0])
            self.sell()
            print('OPEN SHORT, %.2f' % self.data.close[0])

In [140]:
cerebro = bt.Cerebro()

In [141]:
cerebro.adddata(feed)

In [142]:
cerebro.addstrategy(EMACross)
cerebro.broker.set_cash(100)
cerebro.broker.setcommission(0.0004)
cerebro.addsizer(bt.sizers.PercentSizer, percents = 100)
print('<START> Brokerage account: $%.2f' % cerebro.broker.getvalue())
teststrat = cerebro.run()
print('<FINISH> Brokerage account: $%.2f' % cerebro.broker.getvalue())

<START> Brokerage account: $100.00
OPEN LONG, 2563.86
OPEN SHORT, 2555.97
CLOSE SHORT, 2568.19
OPEN LONG, 2568.19
OPEN SHORT, 2557.09
CLOSE SHORT, 2561.45
OPEN LONG, 2561.45
OPEN SHORT, 2535.75
CLOSE SHORT, 2324.61
OPEN LONG, 2324.61
CLOSE LONG, 2374.05
OPEN SHORT, 2374.05
CLOSE SHORT, 2390.67
OPEN LONG, 2390.67
CLOSE LONG, 2362.15
OPEN SHORT, 2362.15
CLOSE SHORT, 2398.64
OPEN LONG, 2398.64
CLOSE LONG, 2356.62
OPEN SHORT, 2356.62
CLOSE SHORT, 2337.94
OPEN LONG, 2337.94
CLOSE LONG, 2301.34
OPEN SHORT, 2301.34
CLOSE SHORT, 2364.99
OPEN LONG, 2364.99
CLOSE LONG, 2273.65
OPEN SHORT, 2273.65
CLOSE SHORT, 2157.05
OPEN LONG, 2157.05
CLOSE LONG, 2097.46
OPEN SHORT, 2097.46
CLOSE SHORT, 1916.24
OPEN LONG, 1916.24
CLOSE LONG, 1896.00
OPEN SHORT, 1896.00
CLOSE SHORT, 1954.45
OPEN LONG, 1954.45
CLOSE LONG, 1936.84
OPEN SHORT, 1936.84
CLOSE SHORT, 1951.53
OPEN LONG, 1951.53
CLOSE LONG, 2060.98
OPEN SHORT, 2060.98
CLOSE SHORT, 2092.34
OPEN LONG, 2092.34
CLOSE LONG, 2065.32
OPEN SHORT, 2065.32
CLOSE 

CLOSE LONG, 1231.42
OPEN SHORT, 1231.42
CLOSE SHORT, 1235.24
OPEN LONG, 1235.24
CLOSE LONG, 1234.33
OPEN SHORT, 1234.33
CLOSE SHORT, 1226.07
OPEN LONG, 1226.07
CLOSE LONG, 1215.20
OPEN SHORT, 1215.20
CLOSE SHORT, 1219.13
OPEN LONG, 1219.13
CLOSE LONG, 1215.28
OPEN SHORT, 1215.28
CLOSE SHORT, 1223.21
OPEN LONG, 1223.21
CLOSE LONG, 1220.53
OPEN SHORT, 1220.53
CLOSE SHORT, 1205.51
OPEN LONG, 1205.51
CLOSE LONG, 1193.63
OPEN SHORT, 1193.63
CLOSE SHORT, 1186.28
OPEN LONG, 1186.28
CLOSE LONG, 1204.46
OPEN SHORT, 1204.46
CLOSE SHORT, 1112.84
OPEN LONG, 1112.84
CLOSE LONG, 1103.43
OPEN SHORT, 1103.43
CLOSE SHORT, 1032.53
OPEN LONG, 1032.53
CLOSE LONG, 1022.45
OPEN SHORT, 1022.45
CLOSE SHORT, 1030.30
OPEN LONG, 1030.30
CLOSE LONG, 1019.74
OPEN SHORT, 1019.74
CLOSE SHORT, 1027.97
OPEN LONG, 1027.97
CLOSE LONG, 1021.03
OPEN SHORT, 1021.03
CLOSE SHORT, 1028.53
OPEN LONG, 1028.53
CLOSE LONG, 1053.99
OPEN SHORT, 1053.99
CLOSE SHORT, 1068.64
OPEN LONG, 1068.64
CLOSE LONG, 1052.51
OPEN SHORT, 1052.51
